In [30]:
from TranscriptionDataset.ArrangementDataset import ArrangementDataModule
from TranscriptionModel.ArrangementModel import ArrangementModel
from pytorch_lightning import Trainer,seed_everything
import torch
import matplotlib.pyplot as plt
from pytorch_lightning.loggers import WandbLogger
import numpy as np
import os
from tqdm import tqdm

plt.rcParams['figure.figsize'] = [12, 8]
print(f"Cuda : {torch.cuda.is_available()}")
torch.set_float32_matmul_precision('medium')

seed_everything(42, workers=True)
# os.environ['WANDB_MODE'] = 'offline'

Global seed set to 42


Cuda : True


42

In [2]:
SAMPLE_RATE = 16000
dataset = "Trainsets/massive_test2.hdf5"

In [4]:
data_module = ArrangementDataModule(
    location=dataset,
    sample_rate=SAMPLE_RATE,
    val_size=0.1,
    num_workers=10,
    batch_size=4,
    disableOhe=True
)

In [5]:
model = ArrangementModel(
    arrangement_size=6
)

In [6]:
# train
wandb_logger = WandbLogger(
    project="ArrangementModelMSE"
)
trainer = Trainer(
    # profiler="simple",
    default_root_dir="model_weights/",
    accelerator="gpu",
    devices=-1,
    max_epochs=10,
    logger=wandb_logger
)

wandb: Currently logged in as: draguve. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(model, data_module)

In [6]:
trainer.save_checkpoint(r"C:\Users\ritwi\Github\MusicTranscription\model_weights\ArrangementModel_Mini_60s_10epoch_over.ckpt")

In [31]:
model = model.load_from_checkpoint(r"C:\Users\ritwi\Github\MusicTranscription\model_weights\ArrangementModel_Mini_60s_10epoch_over.ckpt")
model.eval()
data_module.setup("")
dataloader = data_module.val_dataloader()
expected = np.array([[0]*6])
model_output = np.array([[0]*6])
for batch in tqdm(enumerate(iter(dataloader))):
    section, tuning, arrangement = batch[1]
    output = model(section).squeeze(1).cpu().detach().numpy()
    arrangement = arrangement.cpu().detach().numpy()
    expected = np.vstack((expected, arrangement))
    model_output = np.vstack((model_output, output))

249it [00:48,  5.09it/s]


In [33]:
print(expected.shape)
print(model_output.shape)
print(np.sum(expected[:,0]))
model_output[:,0]>0

(983, 6)
(983, 6)
972.0


array([False,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,